## Old meta agent

In [1]:
from typing import List
from pydantic import BaseModel, Field

# Pydantic Schemas
class AgentCreationResponse(BaseModel):
    thinking_process: str = Field(..., description="The agent's thinking process and steps to create the next agent.")
    name: str = Field(..., description="The name of the next agent.")
    role: str = Field(..., description="The role of the next agent.")
    function: str = Field(..., description="The function of the next agent.")

# Agent Class
class Agent:
    def __init__(self, name: str, role: str, function: str, llm, schema):
        self.name = name
        self.role = role
        self.function = function
        self.llm = llm
        self.schema = schema

    def run(self, task: str) -> BaseModel:
        # Construct the system prompt
        system_prompt = f"""You are {self.name}.

Role:
{self.role}

Function:
{self.function}
"""

        if self.schema == AgentCreationResponse:
            # User prompt for creating the next agent
            user_prompt = f"""Task:
{task}

Please create the next agent by providing the following in JSON format:

{{
    "name": "Provide a suitable name for the next agent.",
    "role": "Define the role of the next agent in detail, making it significantly more specific and comprehensive than your own role.",
    "function": "Describe the function of the next agent thoroughly, elaborating on methodologies, strategies, considerations, potential challenges, and how to address them to solve the task effectively."
}}

Ensure that the next agent's role and function are more detailed and stronger than your own. Think critically about how the next agent can be better equipped to solve the task, and include any improvements, enhancements, or additional capabilities that would assist in achieving the best possible outcome. Be creative and think out of the box and always prompt the next agent to be more specific and detailed and make it think step by step and various steps to solve the task effectively.
"""
        else:
            # User prompt for the final agent to solve the task
            user_prompt = f"""Task:
{task}

Please provide a detailed answer to the task, following your role and function. Ensure that your response is comprehensive, accurate, and addresses all aspects necessary to solve the task effectively.
"""

        # Generate the response using the LLM
        prompt = self.llm.format_prompt(system_prompt=system_prompt, user_prompt=user_prompt)
        result = self.llm.generate(prompt, schema=self.schema)
        return result

# MetaAgentSystem Class
class MetaAgentSystem:
    def __init__(self, llm):
        self.llm = llm

    def create_meta_agents(self, n: int, initial_task: str, answer_schema: BaseModel):
        agents = []
        task = initial_task

        for level in range(n):
            if level == 0:
                # Initial agent details
                agent_name = f"Agent Level {level + 1}"
                role = "Your role is to create an agent that is better suited to solve the given task."
                function = "Your function is to define the name, role, and function of the next agent, making it more specific and detailed to solve the task."
            else:
                # Update the agent's details from the previous agent's output
                agent_name = agent_creation_result.name
                role = agent_creation_result.role
                function = agent_creation_result.function

            if level < n - 1:
                # Create an agent that will create the next agent
                agent = Agent(agent_name, role, function, self.llm, AgentCreationResponse)
                agent_creation_result = agent.run(task)

                # Store the agent's details
                agents.append({
                    "level": level + 1,
                    "name": agent_name,
                    "role": role,
                    "function": function
                })

                # Print out the agent's details
                self.print_agent_details(level + 1, agent_name, role, function, agent_creation_result)
            else:
                # Final agent to solve the task
                final_agent = Agent(agent_name, role, function, self.llm, answer_schema)
                final_result = final_agent.run(task)

                # Print the final agent's details and response
                print(f"Final Agent's Name: {agent_name}")
                print(f"Final Agent's Role: {role}")
                print(f"Final Agent's Function: {function}")
                print(f"\n{agent_name}'s Response:\n{final_result}\n")

                return final_result

    @staticmethod
    def print_agent_details(level, name, role, function, agent_response):
        print(f"Agent Level {level}")
        print(f"Name: {name}")
        print(f"Role: {role}")
        print(f"Function: {function}")
        print("Agent's output:")
        print(agent_response.model_dump_json(indent=2))
        print("\n")

## Second new improved meta agent

In [34]:
from typing import List
from pydantic import BaseModel, Field

# Pydantic Schemas
class AgentCreationResponse(BaseModel):
    thinking_process: str = Field(..., description="The agent's thinking process and steps to create the next agent.")
    name: str = Field(..., description="The name of the next agent.")
    role: str = Field(..., description="The role of the next agent.")
    function: str = Field(..., description="The function of the next agent.")

# Agent Class
class Agent:
    def __init__(self, name: str, role: str, function: str, llm, schema):
        self.name = name
        self.role = role
        self.function = function
        self.llm = llm  # The LLM instance
        self.schema = schema  # The expected output schema

    def run(self, task: str) -> BaseModel:
        # Construct the system prompt
        system_prompt = f"""You are {self.name}.

Role:
{self.role}

Function:
{self.function}
"""

        if self.schema == AgentCreationResponse:
            # User prompt for creating the next agent
            user_prompt = f"""Task:
{task}

As {self.name}, your goal is to create an agent that is better at creating agents that can solve the given task in general. Think about how to enhance the capabilities of the next agent to improve its ability to create effective agents for solving the problem.

Please create the next agent by providing the following in JSON format:

{{
    "thinking_process": "Explain your thinking process and steps to create the next agent.",
    "name": "Provide a suitable name for the next agent.",
    "role": "Define the role of the next agent in detail, focusing on improving the ability to create agents that can solve the problem.",
    "function": "Describe the function of the next agent thoroughly, elaborating on strategies, methodologies, considerations, and how to address challenges in creating agents capable of solving the task effectively."
}}

Ensure that the next agent's role and function are more advanced and effective than your own. Be creative and think critically about how to improve the agent creation process, making the next agent better at creating agents for solving the task.This task will be solved by a Large Language Model based system. Agents do not have any tools or other things to access you just have reasoning skills to think and answer the task.
"""
        else:
            # User prompt for the final agent to solve the task
            user_prompt = f"""Task:
{task}

As {self.name}, please provide a detailed answer to the task, following your role and function. Ensure that your response is comprehensive, accurate, and addresses all aspects necessary to solve the task effectively. This task will be solved by a Large Language Model based system. Agents do not have any tools or other things to access you just have reasoning skills to think and answer the task.
"""

        # Generate the response using the LLM
        prompt = self.llm.format_prompt(system_prompt=system_prompt, user_prompt=user_prompt)
        result = self.llm.generate(prompt, schema=self.schema)
        return result

# MetaAgentSystem Class
# MetaAgentSystem Class with Modified Final Agent Creation
class MetaAgentSystem:
    def __init__(self, llm):
        self.llm = llm  # The LLM instance

    def create_meta_agents(self, n: int, initial_task: str, answer_schema: BaseModel):
        agents = []
        task = initial_task

        for level in range(n):
            if level == 0:
                # Initial agent details
                agent_name = f"Meta Agent Level {level + 1}"
                role = "Your role is to create an agent that can create agents better suited to solve the given task."
                function = "Your function is to define the name, role, and function of the next agent, focusing on enhancing the ability to create agents that can effectively solve the problem. This task will be solved by a Large Language Model based system. Agents do not have any tools or other things to access you just have reasoning skills to think and answer the task."
            else:
                # Update the agent's details from the previous agent's output
                agent_name = agent_creation_result.name
                role = agent_creation_result.role
                function = agent_creation_result.function

            if level < n - 1:
                # Create an agent that will create the next agent
                agent = Agent(agent_name, role, function, self.llm, AgentCreationResponse)
                agent_creation_result = agent.run(task)

                # Store the agent's details
                agents.append({
                    "level": level + 1,
                    "name": agent_name,
                    "role": role,
                    "function": function,
                    "thinking_process": agent_creation_result.thinking_process
                })

                # Print out the agent's details
                self.print_agent_details(level + 1, agent_name, role, function, agent_creation_result)
            else:
                # In the final step, instead of solving, this agent will create an agent to solve the problem
                final_agent_creation = Agent(agent_name, role, function, self.llm, AgentCreationResponse)
                final_agent_result = final_agent_creation.run(task)
                
                # Use the final agent that was created by the n-th agent
                final_agent_name = final_agent_result.name
                final_agent_role = final_agent_result.role
                final_agent_function = final_agent_result.function

                # Now, create the final agent that solves the problem
                final_agent = Agent(final_agent_name, final_agent_role, final_agent_function, self.llm, answer_schema)
                final_result = final_agent.run(task)

                # Print the final agent's details and response
                print(f"Final Agent's Name: {final_agent_name}")
                print(f"Final Agent's Role: {final_agent_role}")
                print(f"Final Agent's Function: {final_agent_function}")
                print(f"\n{final_agent_name}'s Response:\n{final_result}\n")

                return final_result

    @staticmethod
    def print_agent_details(level, name, role, function, agent_response):
        print(f"Agent Level {level}")
        print(f"Name: {name}")
        print(f"Role: {role}")
        print(f"Function: {function}")
        print("Agent's output:")
        print(agent_response.model_dump_json(indent=2))
        print("\n")

In [35]:
import sys;sys.path.append('../')
from multi_agent_llm import OpenAILLM
import os

from rich import print

openai_key=open('../openai.key').read().strip()
os.environ['OPENAI_API_KEY'] = openai_key



llm = OpenAILLM(model_name="gpt-4o-mini")

In [40]:
meta_agent_system = MetaAgentSystem(llm)

# Define the initial task
initial_task = """How many R's are there in  the word "STRAWBERRY"?"""

# Define the answer schema
class Answer(BaseModel):
    explanation: str = Field(..., description="Detailed explanation for the solution.")
    answer: str = Field(..., description="final answer")

# initial_task = """
# Answer the following multiple-choice question. Provide your final answer as 'A', 'B', 'C', or 'D', followed by a brief explanation.\n\nQuestion: Particles are collided at the center of a spherical detector producing new type of particles that travel uninterrupted at ultra-relativistic velocities highly centered around Lorentz factor of ~20. On average, one third of these fast-decaying particles reaches the detector inner walls. \nThe radius of the detector is 30 meters.\n\nWhat Lorentz factor is needed in order to have about two thirds of these particles reaching the detector inner walls?\n\nOptions:\nA. 54\nB. 40\nC. 28\nD. 68\n\nAnswer (A/B/C/D/Uncertain)
        
# """


# Set the number of meta-agent levels
n = 2

# Run the meta-agent system
final_result = meta_agent_system.create_meta_agents(n, initial_task, Answer)

# Print the final result
print("Final Result:")
print(final_result.explanation)
print(final_result.answer)

Agent Level 1

Name: Meta Agent Level 1

Role: Your role is to create an agent that can create agents better suited to solve the given task.

Function: Your function is to define the name, role, and function of the next agent, focusing on enhancing the 
ability to create agents that can effectively solve the problem. This task will be solved by a Large Language Model
based system. Agents do not have any tools or other things to access you just have reasoning skills to think and 
answer the task.

Agent's output:

...

Final Agent's Function: The CountArchitect functions by first conducting an analysis of the input data (textual 
content) to locate specific counting tasks, such as identifying the frequency of specific characters. It will 
employ advanced pattern recognition methodologies, allowing it to understand language variants and user 
preferences. By utilizing a feedback loop mechanism, it can continually refine the algorithms based on the accuracy
of previous agents' outputs and user feedback, making it increasingly adept at creating agents for counting tasks. 
Additionally, it will incorporate user interaction protocols that enable it to ask clarifying questions to better 
understand the specifics of counting tasks before proceeding, ensuring accuracy and relevance in its output.

CountArchitect's Response:
explanation="To find out how many occurrences of the letter 'R' exist in the word 'STRAWBERRY', we can carefully 
analyze the word letter by letter. The word 'STRAWBERRY' consists of the following letters: S, T, R, A, W, B, E, R,
R, Y. When we observe each letter, we can count the instances of 'R'. In this case, the letter 'R' appears three 
times: once in the third position, once in the seventh position, and once in the eighth position of the word. 
Therefore, the total count of 'R's is 3." answer='3'

Final Result:

To find out how many occurrences of the letter 'R' exist in the word 'STRAWBERRY', we can carefully analyze the 
word letter by letter. The word 'STRAWBERRY' consists of the following letters: S, T, R, A, W, B, E, R, R, Y. When 
we observe each letter, we can count the instances of 'R'. In this case, the letter 'R' appears three times: once 
in the third position, once in the seventh position, and once in the eighth position of the word. Therefore, the 
total count of 'R's is 3.

3